# Instructions

* Due date is 11/15 midnight.
* Please check this original colab often to see if anything changes in the instructions. Last updated: 11/13 4:20pm.
* Make a copy of this colab, add your code, run all the cells and save output. Submit your .ipynb file with output. Don't submit a link to your colab, you must submit actual .ipynb file with output.
* You must train the model and show the test output. Points will be deducted if your model is not trained / produces run time errors. If you try to build a very large model and it doesn't finish training, than you would get points deducted, so make reasonable adjustments to keep the model size under check, do not train very small models, do not train very large models.


Q1 Trees (10 Points)

1. Implement a simple linear regressor over the housing dataset. Measure MSE loss on the test set.

2. Implement a gradient boosted regressor over the same training set, and measure its MSE loss over the test set.



Part 1


In [1]:
from packaging import version
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import joblib

np.random.seed(1237)

# especially with joblib since I haven't used it in a while.
#I follow the housing model we use in class and slides.
#There wasn't much issue but I did run into a function problem where I can't load the data set



Part 2

In [2]:
from pathlib import Path
import urllib
import tarfile

def load_housing_data():
    tarball = Path("datasets/housing.tgz")
    #Here i am doing is it first check if the housing.tgz is already downloaded.
    # if not then downloaded
    if not tarball.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball)
      #here i want you and I to understand Im using panda to read data into data frame
        with tarfile.open(tarball) as tar:
            tar.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

data = load_housing_data()
data.head() #beta test

#if you see a msitake in the file path it was my typo I did earlier in the module to ensure the paths worked now.



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Part 3

In [3]:
data['ocean_proximity'] = data['ocean_proximity'].astype(str)
encoder = OneHotEncoder()
encoded_proximity = encoder.fit_transform(data[['ocean_proximity']]).toarray()
proximity_df = pd.DataFrame(encoded_proximity, columns=encoder.get_feature_names_out(['ocean_proximity']))

data = pd.concat([data, proximity_df], axis=1).drop('ocean_proximity', axis=1)

# feature set
target = 'median_house_value'
feature_names = data.columns.difference([target])

# very long time to findout OneHotEncoder outputs numpy, and I was trying to add it directly to the DataFrame.
#big mistake but needed to wrap it in a DataFrame to add col name


Part 4

In [4]:
# splitting like always
train_data, test_data = train_test_split(data, test_size=0.2, random_state=1237)
X_train = train_data[feature_names]
y_train = train_data[[target]]
X_test = test_data[feature_names]
y_test = test_data[[target]]

#median each column
imputer = SimpleImputer(strategy='median')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=feature_names)
X_test = pd.DataFrame(imputer.transform(X_test), columns=feature_names)



Part 5 Nov/15

In [5]:
# Training the Linear Regression model

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
# Evaluating
lin_mse = mean_squared_error(y_test, lin_reg.predict(X_test))
print("L Regression MSE:", lin_mse)

#it threw a ValueError.
# Once I added the simpleimput step, works great and I was able to calculate the MSE.


L Regression MSE: 4620773668.969507


Part 6

In [6]:
#GBR model
gb_reg = GradientBoostingRegressor(random_state=1237)
gb_reg.fit(X_train, y_train.values.ravel())
gb_mse = mean_squared_error(y_test, gb_reg.predict(X_test))
print("GBR MSE :", gb_mse)

# When training this model, I accidentally passed the labels with flat, which caused an error.
#Remember always use values ravel! to get a 1D array.

#https://www.geeksforgeeks.org/numpy-ravel-python/


GBR MSE : 2963016650.5902896
